In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv('Data_Q2.csv')
df.head()
scalar=StandardScaler()
df_std=scalar.fit_transform(df)
df=pd.DataFrame(df_std,columns=df.columns)
df

Temperature  Humidity  Wind Speed      Flow  Consumption
0      -1.931878  1.404846   -0.641791 -0.664370    -3.450208
1       0.407654 -0.874474   -0.640417 -0.665724    -3.282442
2       0.459476 -0.888013   -0.640875 -0.665229    -3.255953
3       0.433565 -0.807575   -0.641333 -0.665412    -3.185315
4       0.393521 -0.610863   -0.641333 -0.664266    -3.167655
..           ...       ...         ...       ...          ...
995     0.836365 -2.637721    1.572655  0.152039     1.926266
996    -1.594563  0.082809    1.574029  1.046430     1.935127
997     0.242766  0.552690    1.570823 -0.665047     1.935127
998    -0.397944  0.162450   -0.645913 -0.666479     1.935127
999     0.683254 -0.831468   -0.643165  2.257439     1.935666

[1000 rows x 5 columns]

2a)

$min_{\beta}   \ \lambda \| \beta \|{2}^2 + \| y- X\beta \|_2^2 = min{\beta} \ Z \ \  (say)$

let X is of order N x D

$Z = \lambda \| \beta \|_{2}^2 + \| y- X\beta \|_2^2 \\ \Rightarrow \lambda\beta^T \beta   + (y-X\beta)^T(y-X\beta)
\\ \Rightarrow y^T y - \beta ^T X^T y + \beta^T X ^T X \beta - y^T X \beta + \lambda \beta^T \beta$

To find the optimal weight vector $\boldsymbol{\beta}$, we take the derivative of the cost function with respect to $\boldsymbol{\beta}$ and set it to zero

$\Rightarrow 2X^T X \beta -2X^Ty +2\lambda \beta =0 \\ \Rightarrow X^TX\beta + \lambda \beta = X^Ty \ \  \ \ \ \  ....(1)$
 $ or X^TX\beta -X^Ty=-\lambda \beta  
\\ \Rightarrow \beta = \frac{1}{\lambda} X^T(y-X\beta) \\ so \  \beta \  can \  be  \ written \  as \\ \beta= X^T \alpha \ \ \ \   ...(2) \ where \ \ \alpha = \frac{1}{\lambda} (y-X\beta) \\ 
now \   we  \ can \  write \  (1) \  as \  
 \\ \beta=(X^T X + \lambda I_{D})^{-1} X^T y
 \\ \Rightarrow \beta = X^T(X X^T + \lambda I_{N})^{-1}y  \ \ \ ..(3) \  by \  matrix  \ inversion \  lemma \\
  \ comparing  \ (2)  \  and \  (3)  \ we \ got \ \\ \alpha = (X X^T + \lambda I_{N})^{-1}y \\ now  \ in \  this \  relation  \ X^TX  \ \text{can be effectively replace by a kernel matrix using the kernel function let k as kernel } \\ 
  \alpha = (K + \lambda I_N)^{-1}y
  \\ \text{then we can rewrite} \ \beta \ as \ 
  \\ \beta=X^T \alpha = \sum_{i=1}^{i=N} 
\alpha_i X_i \\   \text{This tells us that the solution vector is just a linear sum of the N training vectors. When we plug this in at test time to compute the predictive mean},\\ \text{let's compute for a sample} \  x̃  \\  \\  ỹ(x̃) = \beta^Tx̃ = \sum_{i=1}^{i=N} \alpha_i x_i^Tx̃ = \sum_{i=1}^{i=N} \alpha_i K(x̃,x_i)$

In [ ]:
df.describe()

Temperature      Humidity    Wind Speed         Flow   Consumption
count  1.000000e+03  1.000000e+03  1.000000e+03  1000.000000  1.000000e+03
mean   1.989520e-16  1.563194e-16  7.460699e-17     0.000000 -1.591616e-15
std    1.000500e+00  1.000500e+00  1.000500e+00     1.000500  1.000500e+00
min   -2.171202e+00 -2.908500e+00 -6.504934e-01    -0.668041 -3.450208e+00
25%   -7.400875e-01 -7.560080e-01 -6.427073e-01    -0.665516 -6.790723e-01
50%    1.897660e-01  2.420906e-01 -6.408753e-01    -0.524340  1.096184e-01
75%    7.786535e-01  8.871812e-01  1.570823e+00     0.308264  7.168465e-01
max    2.468761e+00  1.540236e+00  1.609753e+00     6.407833  1.935666e+00

In [ ]:
from sklearn.model_selection import train_test_split
x_value, y_value= train_test_split(df, test_size=0.2, random_state=2000)
print('Standard deviation of the in x_value is ', x_value['Consumption'].std())
print('Standard deviation of  the y_value', y_value['Consumption'].std())
print(x_value['Consumption'].describe())
print(y_value['Consumption'].describe())
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
import numpy as np
gamma_range = np.logspace(-5, 5, 11)
param_grid = {'gamma': gamma_range}
kr = KernelRidge(kernel='rbf')
kr_cv = GridSearchCV(kr, param_grid=param_grid, cv=5)
kr_cv.fit(x_value.drop(columns=['Consumption']), x_value['Consumption'])
print(' the Best gamma are:', kr_cv.best_params_['gamma'])
print('Best_score:', kr_cv.best_score_)

Standard deviation of the in t1 is  1.0072653454448557
Standard deviation of  the t2 0.9742226367253248
count    800.000000
mean      -0.010897
std        1.007265
min       -3.282442
25%       -0.704147
50%        0.106082
75%        0.722254
max        1.935127
Name: Consumption, dtype: float64
count    200.000000
mean       0.043587
std        0.974223
min       -3.450208
25%       -0.588909
50%        0.199816
75%        0.690958
max        1.935666
Name: Consumption, dtype: float64
 the Best gamma are: 10.0
Best_score: 0.21496230751907994


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
y_train_pred = kr_cv.predict(x_value.drop(columns=['Consumption']))
rmse_train = np.sqrt(mean_squared_error(x_value['Consumption'], y_train_pred))
r2_train = r2_score(x_value['Consumption'], y_train_pred)
print('RMSE on x_value set:', rmse_train)
print('R2 on y_value set:', r2_train)
y_test_pred = kr_cv.predict(y_value.drop(columns=['Consumption']))
rmse_test = np.sqrt(mean_squared_error(y_value['Consumption'], y_test_pred))
r2_test = r2_score(y_value['Consumption'], y_test_pred)
print('RMSE on x_value set:', rmse_test)
print('R2 on y_value set:', r2_test)

RMSE on T1 set: 0.6421990252031164
R2 on T1 set: 0.5929997211837872
RMSE on x_value set: 0.8620622526160943
R2 on y_value set: 0.21306702477088502


Based on the provided information, it seems like you have trained a regression model and evaluated its performance on two different sets - T1 set and x_value set.

The RMSE (Root Mean Squared Error) is a measure of the difference between the predicted values and the actual values, with lower values indicating better performance.

On the T1 set, your model has an RMSE of 0.642 and an R2 score of 0.593. An R2 score of 1.0 indicates a perfect fit, while a score of 0 indicates that the model is no better than predicting the mean of the target variable. Therefore, an R2 score of 0.593 indicates that your model has moderate predictive power on the T1 set.

On the x_value set, your model has a higher RMSE of 0.862 and a lower R2 score of 0.213. This suggests that your model may not be performing as well on the x_value set, as indicated by the higher error and lower R2 score.

It's difficult to provide specific recommendations without more context on the problem you are trying to solve and the data you are using. However, if you are looking to improve the performance of your model, you could consider various approaches such as feature engineering, trying different models, hyperparameter tuning, or using more data.

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

# Define training and testing data
x_train, y_train = x_value.drop('Consumption', axis=1), x_value['Consumption']
x_test, y_test = y_value.drop('Consumption', axis=1),y_value['Consumption']

# Perform k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=200)
lr = LinearRegression()
rmse_train_scores, r2_train_scores = [], []
rmse_test_scores, r2_test_scores = [], []
for train_index, val_index in kf.split(x_train):
    x_train_fold, y_train_fold = x_train.iloc[train_index], y_train.iloc[train_index]
    x_val_fold, y_val_fold = x_train.iloc[val_index], y_train.iloc[val_index]
    lr.fit(x_train_fold, y_train_fold)
    y_train_pred = lr.predict(x_val_fold)
    y_test_pred = lr.predict(x_test)
    rmse_train = np.sqrt(mean_squared_error(y_val_fold, y_train_pred))
    r2_train = r2_score(y_val_fold, y_train_pred)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)
    rmse_train_scores.append(rmse_train)
    r2_train_scores.append(r2_train)
    rmse_test_scores.append(rmse_test)
    r2_test_scores.append(r2_test)

# Print results
print("Average RMSE on training set:", np.mean(rmse_train_scores))
print("Average R2 on training set:", np.mean(r2_train_scores))
print("Average RMSE on testing set:", np.mean(rmse_test_scores))
print("Average R2 on testing set:", np.mean(r2_test_scores))


Average RMSE on training set: 0.9399709140439946
Average R2 on training set: 0.12068450136886084
Average RMSE on testing set: 0.9255841565372757
Average R2 on testing set: 0.09281780118882835


Based on the provided metrics, it seems that the model is not performing very well. The root mean squared error (RMSE) values for both the training and testing sets are relatively high, indicating that the model's predictions are not very accurate. Additionally, the R-squared (R2) values are very low, suggesting that the model does not explain much of the variance in the data.

Overall, it appears that the model may be underfitting, meaning that it is not capturing the complexity of the data and is therefore not able to make accurate predictions. To improve the model's performance, it may be necessary to consider alternative model architectures, feature engineering, or hyperparameter tuning. Additionally, it may be helpful to gather more data or address any issues with data quality that could be impacting the model's performance

In [ ]:
frame2 = pd.read_csv('Data_Q2.csv')
conditions = [
    (frame2['Consumption'] <= 6500),
    (frame2['Consumption'] > 6500) & (frame2['Consumption'] <= 7000),
    (frame2['Consumption'] > 7000) & (frame2['Consumption'] <= 7500),
    (frame2['Consumption'] > 7500) & (frame2['Consumption'] <= 8000),
    (frame2['Consumption'] > 8000) & (frame2['Consumption'] <= 8500),
    (frame2['Consumption'] > 8500) & (frame2['Consumption'] <= 9000),(frame2['Consumption'] > 9000)
]
classes = [1, 2, 3, 4, 5, 6, 7]
frame2['Class'] = np.select(conditions, classes)
frame2.head()

Temperature  Humidity  Wind Speed   Flow  Consumption  Class
0        5.578     93.00       0.082  0.185  5935.174070      1
1       15.510     64.38       0.085  0.133  6044.657863      1
2       15.730     64.21       0.084  0.152  6061.944778      1
3       15.620     65.22       0.083  0.145  6108.043217      1
4       15.450     67.69       0.083  0.189  6119.567827      1

In [ ]:
classes = np.unique(frame2['Class'])
number_class = len(classes)
print(" classes", number_class)
for c in classes:
    num_samples = np.sum(frame2['Class'] == c)
    print("Class", c, num_samples, "samples value of are ")
cols = frame2.columns[:-1]
scaler = StandardScaler()
for i in range(1, 8):
    mask = (frame2['Class'] == i)
    frame2.loc[mask, cols] = scaler.fit_transform(frame2.loc[mask, cols])
frame2.head()

 classes 7
Class 1 15 samples value of are 
Class 2 27 samples value of are 
Class 3 103 samples value of are 
Class 4 218 samples value of are 
Class 5 280 samples value of are 
Class 6 255 samples value of are 
Class 7 102 samples value of are 


Temperature  Humidity  Wind Speed      Flow  Consumption  Class
0    -1.690575  1.563325   -0.508727 -0.365626    -1.925782      1
1     1.083295 -1.525195    0.581402 -0.412271    -1.304210      1
2     1.144738 -1.543541    0.218026 -0.395227    -1.206067      1
3     1.114017 -1.434547   -0.145350 -0.401507    -0.944353      1
4     1.066538 -1.167998   -0.145350 -0.362038    -0.878924      1

In [ ]:
x = frame2.loc[x_value.index]
y = frame2.loc[y_value.index]
X_train = x.drop(['Class', 'Consumption'], axis=1)
y_train = x['Class']
X_test = y.drop(['Class', 'Consumption'], axis=1)
y_test = y['Class']
param_grid = {"gamma": [0.1, 1, 10]}
svc = SVC(kernel="rbf", class_weight="balanced")
svm_model = GridSearchCV(svc, param_grid, cv=5)
svm_model.fit(X_train, y_train)
models = []
for i in range(1,8):
    mask = y_train == i
    X_train_i = X_train[mask]
    y_train_i = x[mask]["Consumption"]
    param_grid = {"gamma": [0.1, 1, 10]}
    krr = KernelRidge(kernel="rbf")
    model = GridSearchCV(krr, param_grid, cv=5)
    model.fit(X_train_i, y_train_i)
    models.append(model)
def y_value_predited(x):  
  y_pred = svm_model.predict(x)
  y_pred_consumption = []
  for j,i in enumerate(y_pred):
      model = models[i-1]
      y_pred_i = model.predict(x.iloc[j].values.reshape(1, -1))
      y_pred_consumption.append(y_pred_i[0])
  return y_pred_consumption
rmse_t3 = mean_squared_error(x["Consumption"],y_value_predited(X_train)) ** 0.5
rmse_t4 = mean_squared_error(y["Consumption"],y_value_predited(X_test)) ** 0.5
print(' THE RMSE  OF  x :',rmse_t3)
print('THE RMSE OF  y :',rmse_t4)

 THE RMSE  OF  x : 0.6928383193024183
THE RMSE OF  y : 0.9971875610243701


However, in general, if the RMSE value for the x train set is significantly smaller than the RMSE value for the x test set, it may indicate that the model is overfitting the training data and not generalizing well to new, unseen data. In this case, it may be necessary to adjust the model or its hyperparameters to improve its generalization performance.

On the other hand, if the RMSE values for the train and test sets are relatively close to each other, it may indicate that the model is well-balanced and can generalize well to new data.

It's also worth noting that the RMSE values should be interpreted in the context of the specific problem and the units of the target variable being predicted. A smaller RMSE value may not always necessarily indicate better predictive performance, depending on the specific goals of the model and the characteristics of the data.